# **CSI 382 - Data Mining and Knowledge Discovery**

# **Lab 4 - Statistical Approaches to Estimation and Prediction**

If estimation and prediction are considered to be data mining tasks, statistical
analysts have been performing data mining for over a century. In this lab we
will be examining :
* univariate methods
* statistical estimation
* prediction methods

These methods include point estimation and confidence interval estimation.
Next we will consider simple linear regression, where the relationship between
two numerical variables is investigated. Finally, we will examine multiple
regression, where the relationship between a response variable and a set of
predictor variables is modeled linearly.

# **Dataset for Lab 4**



Nutritionists are concerned that people have a good breakfast. But what does that mean? students collected nutrition information from the nutrition labels of cereals in one supermarket.

You can find the dataset in this [link](https://drive.google.com/file/d/1FZ83g_MH6tGOziXZwGMThD6Ie_zsibj4/view?usp=sharing).

## **Dataset Description**

Fields in the dataset:

* Name: Name of cereal
* mfr: Manufacturer of cereal
    * A = American Home Food Products;
    * G = General Mills
    * K = Kelloggs
    * N = Nabisco
    * P = Post
    * Q = Quaker Oats
    * R = Ralston Purina
* type:
    * cold
    * hot
* calories: calories per serving
* protein: grams of protein
* fat: grams of fat
* sodium: milligrams of sodium
* fiber: grams of dietary fiber
* carbo: grams of complex carbohydrates
* sugars: grams of sugars
* potass: milligrams of potassium
* vitamins: vitamins and minerals - 0, 25, or 100, indicating the typical percentage of FDA recommended
* shelf: display shelf (1, 2, or 3, counting from the floor)
* weight: weight in ounces of one serving
* cups: number of cups in one serving
* rating: a rating of the cereals (from Consumer Reports)

## **Loading the dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# load the dataset
df = pd.read_csv('/content/drive/MyDrive/cereal.csv')

#Check number of rows and columns in the dataset
print("The dataset has %d rows and %d columns." % df.shape)

## **Dataset Preprocessing**

In order to continue operations with the dataset we need to remove any missing values that might be present in the dataset. Here, $-1$ is used to indicate missing values.

In [ ]:
missing_values = df.isin([-1]).any(axis=1)

In [ ]:
df[missing_values]

In [ ]:
df = df.replace(-1,df.mean())

# **UNIVARIATE METHODS: MEASURES OF CENTER AND SPREAD**

## **Measures of center and spread**

In [ ]:
df.describe()

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode

configure_plotly_browser_state()

init_notebook_mode(connected=False)# to show plots in notebook

from plotly.offline import plot, iplot

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')


colors = plotly.colors.DEFAULT_PLOTLY_COLORS


def create_box_trace(col, visible=False):
    return go.Box(
        y=df[col],
        name=col,
        marker = dict(color = colors[0]),
        visible=visible,
    )

features_not_for_hist = ["name", "mfr", "type"]
features_for_hist = [x for x in df.columns if x not in features_not_for_hist]
# remove features with too less distinct values (e.g. binary features), because boxplot does not make any sense for them
features_for_box = [col for col in features_for_hist if len(df[col].unique())>5]

active_idx = 0
box_traces = [(create_box_trace(col) if i != active_idx else create_box_trace(col, visible=True)) for i, col in enumerate(features_for_box)]

data = box_traces

n_features = len(features_for_box)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
        label = features_for_box[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ",
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='box_slider')

## **Estimation**

The value of the population mean number of customer service calls $\mu$ is unknown for a variety of reasons, including the fact that the data may not yet have been collected or warehoused. Instead, data analysts would use estimation.
For example, they would estimate the unknown value of the population mean $\mu$ by obtaining a sample and computing the sample mean $\bar{x}$, which would be used to estimate $\mu$.

### **Confidence Interval Estimate- Example**

Usually, finding a large sample size is not a problem for many data mining scenarios. For example, using the statistics in Figure 1, we can find the 95% t-interval for the mean number of sugars for all Cereals is as follows:


In [ ]:
import numpy as np
import scipy.stats as st

st.t.interval(alpha=0.95, df=len(df['sugars'])-1, loc=np.mean(df['sugars']), scale=st.sem(df['sugars']))

We are $95\%$ confident that the population mean number of sugars for all cereals falls between 6.04 and 8.01 grams.

However, data miners are often called upon to estimate the behavior of specific subsets of customers instead of the entire customer base, as in the example above. For example, suppose that we are interested in estimating the mean number
of sugars for cereals made by kellogs. This considerably restricts the sample size, as shown below:

In [ ]:
kellogs = df[df["mfr"]=="K"]

In [ ]:
kellogs['sugars'].describe()

We may find the $95\%$ t-confidence interval estimate as follows:

In [ ]:
st.t.interval(alpha=0.95, df=len(kellogs['sugars'])-1, loc=np.mean(kellogs['sugars']), scale=st.sem(kellogs['sugars']))

We are $95\%$ confident that the mean number of sugar content for all cereals made by kellogs falls between 5.618939147403277 and 9.511495635205419 grams.

# **BIVARIATE METHODS**

So far we have discussed estimation measures for one variable at a time. Ana-
lysts, however, are often interested in bivariate methods of estimation, for example, using the value of one variable to estimate the value of a different variable.

To help us learn about regression methods for estimation and prediction, let us
get acquainted with a new data set, cereals. The cereals dataset can be found at
Data and Story Library website.

The cereals dataset [2] contains nutrition information for 77 breakfast cereals
and includes the following variables:

* Cereal name
* Cereal manufacturer
* Type (hot or cold)
* Calories per serving
* Grams of protein
* Grams of fat
* Milligrams of sodium
* Grams of fiber
* Grams of carbohydrates
* Grams of sugars

* Milligrams of potassium
* Percentage of recommended
* daily allowance of vitamins (0%, 25%, or 100%)
* Weight of one serving
* Number of cups per serving
* Shelf location (1 = bottom, 2 = middle, 3 = top)
* Nutritional rating, calculated by Consumer Reports

Figure 3 provides a peek at the eight of these fields for the first 16 cereals.We are interested in estimating the nutritional rating of a cereal given its sugar content.

In [ ]:
df.head(10)

## **Regression - Example**

Figure 4 shows a scatter plot of the nutritional rating versus the sugar content
for the 77 cereals, along with the least-squares regression line.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(color_codes=True)
fig, ax = plt.subplots()
fig.set_size_inches(18.5, 10.5)
sns.regplot(x="sugars", y="rating", data=df, ax=ax)

## **Regression - Calculation**

To calculate $b_0$ and $b_1$ we need to calculate the $\bar{x}$, $\bar{y}$,$\sum{x^2}$,$\sum{xy}$,$\sigma$ and co-variance of x and y. In the given scenario, $\bar{x}=6.92$, $\bar{y}=42.67$, $\sigma=4.44$, $\sum{x^2} = 5191$, $\sum{xy} = 19135.91$.

Thus we can calculate the variance of $x$ as:

$\sigma(x)  = \sum{x^2}-n\times\bar{x} = (\sigma)^2 = 19.76$

Now we can find the co-variance of $x$ and $y$ by using:

Cov(x,y)  $= \frac{\sum{xy}-n\times\bar{x}\times\bar{y}}{n-1} = -47.43$

slope of the regression line is, $b_1=\frac{Cov(x,y)}{\sigma^{2}(x)} = \frac{-47.43}{19.76} = -2.42$

intercept is, $b_0 = \bar{y} - b_{1}\times\bar{x} = 59.4$

regression equation, $ \hat{y} = 59.4 - 2.42(x)$

In [ ]:
# implementing the regression equation
def regression_equation(x, y):
    x_mean = x.mean()
    y_mean = y.mean()
    x_squared = (x**2).sum()
    xy = (x*y).sum()
    std_dev = x.std()
    var_x = std_dev ** 2
    cov_xy = (xy - len(x)*x_mean*y_mean)/(len(x)-1)
    b_1 = cov_xy/var_x
    b_0 = y_mean - b_1*x_mean
    print('Regression equation is: %.3f %.3f (x)' % (b_0, b_1))
    y_hat = b_0+b_1*x
    return y_hat


## **Residuals**
Now, there is one cereal in our data set that does have a sugar content of 1 gram, Cheerios. Its nutrition rating, however, is 50.765, not 56.98 as we estimated
above for the new cereal with 1 gram of sugar. Cheerios’ point in the scatter
plot is located at $(x = 1, ŷ = 50.765)$, within the oval in Figure 4. Now, the
upper arrow in Figure 4 is pointing to a location on the regression line directly
above the Cheerios point. This is where the regression equation predicted the
nutrition rating to be for a cereal with a sugar content of 1 gram. The prediction
was too high by $56.98 − 50.765 = 6.215$ rating points, which represents the
vertical distance from the Cheerios data point to the regression line. This vertical
distance of $6.215$ rating points, in general $(y − ŷ)$, is known variously as the
prediction error, estimation error, or residual.

In [ ]:
pred_rating = pd.DataFrame()

pred_rating['Cereal'] = df['name']
pred_rating['Actual Rating']  = df['rating']
pred_rating['Predicted Rating'] =  pred
pred_rating['Predicted Error'] = df['rating'] - pred

In [ ]:
pred_rating.head(20)

## **New Prediction**

Suppose that a new cereal called EValley cereal arrives on the market with a very
high sugar content of 30 grams per serving. Let us use our estimated regression
equation to estimate the nutritional rating for the cereal:

$\hat{y} = 59.4 − 2.42(sugars) = 59.4 − 2.42(30) = −13.2$

In other words, EValley’s cereal has so much sugar that its nutritional rating
is actually a negative number, unlike any of the other cereals in the data set
(minimum = 18) and analogous to a student receiving a negative grade on an
exam. What is going on here?

In [ ]:
#plot libaries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode

configure_plotly_browser_state()

init_notebook_mode(connected=False)# to show plots in notebook


layout = dict(
    yaxis=dict(
        title='Rating',
        automargin=True,
    ),
    xaxis=dict(
        title='Feature: sugar content',
        automargin=True,
    ),
)
fig = go.Figure(layout=layout)

# Add traces
fig.add_trace(go.Scatter(x=df['sugars'], y=df['rating'],
                    mode='markers',
                    name='Ratings'))
fig.add_trace(go.Scatter(x=df['sugars'], y=(59.961-2.462*df['sugars']),
                    mode='lines',
                    name='Regression'))


fig.show()

## **MULTIPLE REGRESSION**

Most data mining applications enjoy a wealth (indeed, a superfluity) of data,
with some data sets including hundreds of variables, many of which may have a
linear relationship with the target (response) variable.

Multiple regression modeling provides an elegant method of describing such re-
lationships. Multiple regression models provide improved precision for estima-
tion and prediction, analogous to the improved precision of regression estimates
over univariate estimates.

In [ ]:
#plot libaries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode

configure_plotly_browser_state()

init_notebook_mode(connected=False)# to show plots in notebook

from plotly.offline import plot, iplot

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')

SEED = 17 # specify seed for reproducable results
pd.set_option('display.max_columns', None) # prevents abbreviation (with '...') of columns in prints

colors = plotly.colors.DEFAULT_PLOTLY_COLORS

def create_scatter_trace(x, y, visible=False):
    return go.Scatter(x=df[x], y=df[y],
                    mode='markers',
                    name=y.title(),
                    marker = dict(
                        size=8,
                    ),
                    visible=visible)

def create_line_trace(x,y, visible=False):
    return go.Scatter(x=df[x], y=regression_equation(df[x],df[y]),
                    mode='lines',
                    name='Regression',
                    visible=visible)

features_not_for_reg = ["name", "mfr", "type", "rating"]
features_for_reg = [x for x in df.columns if x not in features_not_for_reg]
active_idx = 0

traces_scatter = [(create_scatter_trace(col,'rating') if i != active_idx else create_scatter_trace(col,'rating', visible=True)) for i, col in enumerate(features_for_reg)]
traces_line = [(create_line_trace(col,'rating') if i != active_idx else create_line_trace(col,'rating', visible=True)) for i, col in enumerate(features_for_reg)]

data = traces_scatter + traces_line

n_features = len(features_for_reg)

steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
        label = features_for_reg[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ",
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='Rating',
        automargin=True,
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='regression_slider')

From Figures 7a and 7b, we would expect that
* protein, fiber, and potassium would be positively correlated with a higher
nutritional rating
* fat, sodium, sugars, and surprisingly, vitamins are negatively correlated
with a higher nutritional rating
* Carbohydrates seem to be uncorrelated with nutritional rating

## **Correlation Coefficients**

We can verify these graphical findings with the correlation coefficients for all
the variables, shown in the Figure 8. The first column (in bold) shows the cor-
relation coefficients of the predictor variables with rating. As expected, protein,
fiber, and potassium are positively correlated with rating, whereas calories, fat,
sodium, and vitamins are negatively correlated.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(14, 10))

corr = df.corr()

ax = sns.heatmap(corr, cmap = "RdBu_r", annot=True)

ax.invert_xaxis()

ax.xaxis.set_ticks_position('top')

plt.title("Heatmap of pairwise correlation of the columns")

plt.show()

Data analysts need to guard against multicollinearity, a condition where some of
the predictor variables are correlated with each other. Multicollinearity leads to
instability in the solution space, leading to possible incoherent results. Even if
such instability is avoided, inclusion of variables that are highly correlated tends
to overemphasize a particular component of the model, since the component
is essentially being double counted. Here, potassium is very highly correlated
with fiber (r = 0.905). Although more sophisticated methods exist for handling
correlated variables, such as principal components analysis, in this introductory
example we simply omit potassium as a predictor.

# **Thats all for today**

# **Tasks**

Go to this [url](https://drive.google.com/file/d/1PU1AQVCgnFL2XbgIJRl3TwkHY5Ag3qIv/view?usp=sharing) and download the data first. In order to know more about the dataset please refer to these links - [UCI/iris](https://archive.ics.uci.edu/ml/datasets/Iris), or [Kaggle/iris_dataset](https://www.kaggle.com/uciml/iris).

**The "species" field refers to the Predicted attribute: class of iris plant.**

 Now try to do the following:

1. Find the measures of center and spread from the dataset.
2. Can you measure your confidence interval estimate for the PetalLengthCm feature?
3. Apply regression on each column against the class field and verify the outcomes using a correlation matrix.